In [1]:
%matplotlib widget

In [2]:
from ipywidgets import interact, interactive
import numpy as np
from scipy import integrate
from matplotlib import pyplot as plt
from matplotlib import ticker
import ipywidgets as widgets

In [3]:
def retire_calc(mc_rounds, c_age, r_age, death, starting_assets, starting_budget, mkt_mean, mkt_std, inflat):
    
    #lists for storing the results of the simulation
    ending_ages = []
    results = []
    markets = []
    results_comb = []
    
    #number of year for each scenario
    years = death - c_age
    
    #constant withdrawal rate, adjusted for inflation
    withdraws = [round(starting_budget * (1 + inflat)** i) for i in range(0, years + 1)]

    #Outer loop for running number or simulation rounds
    for simulation in range(0, mc_rounds):
    
        #create a blank list of ending year asset values with starting asset value
        assets = [0] * years
        assets.insert(0,starting_assets)

    
        random_mkt = np.random.normal(mkt_mean, mkt_std, years).tolist()
    
        #Inner loop through market returns until years reached or assets < 0
        i = 1
    
        #model assumes withdrawal done at the beginning of the year, prior to earnings/losses
        while assets[i - 1] > 0 and i <= years:
            
            #no withdrawals before retirement age
            if i + c_age < r_age:
                assets[i] = round(assets[i-1] * (1 + random_mkt[i-1]))
            
            #at retirement age start withdrawals at the beginning of each year
            if i + c_age >= r_age:
                assets[i] = round((assets[i-1] - withdraws[i]) * (1 + random_mkt[i-1])) 
            
            i += 1
    
        ending_ages.append(c_age + i - 1)
        results.append(assets)
        markets.append(random_mkt)
        results_comb.append((simulation, c_age + i - 1, assets, random_mkt))
        
    line_chart(results, c_age, death)
           
    return results_comb

def line_chart(results, c_age, death):
    
    plt.close()
    fig, ax = plt.subplots()
    ax.plot(range(c_age, death + 1),np.percentile(results, 75, axis=0), label='75% Above Average Market')
    ax.plot(range(c_age, death + 1),np.percentile(results, 50, axis=0), label='50% Average Market')
    ax.plot(range(c_age, death + 1),np.percentile(results, 25, axis=0), label='25% Below Average Market')
    plt.legend()
    ax.set_title('Asset Value by Age and Market Performance')
    ax.set_xlabel('Age')
    ax.set_ylabel('Asset Value')
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("${x:,.0f}"))
    
    plt.show()

def pie_chart(ending_ages):
    
    

In [4]:
#format user selection options
sim_slider = widgets.IntSlider(value=1000,min=100,max=10_000,step=100,description='Simulations:',
                               tooltip='Number of simulations to run')

c_age_slider = widgets.IntSlider(value=45,min=0,max=80,step=1,description='Current Age:',tooltip='Current age')

r_age_slider = widgets.IntSlider(value=65,min=0,max=80,step=1,description='Retire Age:',tooltip='Retirement Age')

death_slider = widgets.IntSlider(value=90,min=81,max=120,step=1,description='Longevity:',tooltip='Expected longevity')

asset_slider = widgets.IntSlider(value=500_000,min=100_000,max=10_000_000,step=10_000,description='Assets:',
                                 tooltip='Starting assets',readout_format='$0,.0f')

budget_slider = widgets.IntSlider(value=50_000,min=10_000,max=250_000,step=1000,description='Budget:',
                                  tooltip='Annual Budget (current dollars)',readout_format='$0,.0f')

mkt_slider = widgets.FloatSlider(value=0.115,min=0,max=1,step=0.001,description='Avg Return:',
                                 tooltip='Average annual market return',readout_format='.1%')

std_slider = widgets.FloatSlider(value=0.195,min=0,max=1,step=0.001,description='Market Std:',
                                 tooltip='Standard deviation of market returns',readout_format='.1%')

inf_slider = widgets.FloatSlider(value=0.03,min=0,max=1,step=0.001,description='Inflation:',
                                 tooltip='Average annual inflation',readout_format='.1%')
#call retire calc function
w = interactive(retire_calc, mc_rounds=sim_slider, c_age=c_age_slider, r_age=r_age_slider, death=death_slider,
                starting_assets=asset_slider, starting_budget=budget_slider, mkt_mean=mkt_slider, mkt_std=std_slider,
                inflat=inf_slider)

#display simulation results
display(w)

interactive(children=(IntSlider(value=1000, description='Simulations:', max=10000, min=100, step=100, tooltip=…